In [164]:
import json
import pprint
import uuid
import xmltodict
import pandas as pd

pd.set_option('display.max_rows', 10)
RESULT_TABLE_NAMES = ['prequal_results', 'xml_details', 'products', 'credit_profiles', 'headers', 'risk_models', 'consumer_identities', 'addr_infos', 'employ_infos', 'trade_lines', 'inquiries', 'info_msgs', 'enhanced_pm_data', 'tradeline_amounts']
with open("./data/prequalresult.json", 'r') as f:
    json_file = json.load(f)
json_file

[{'model': 'common.prequalresult',
  'pk': 1,
  'fields': {'loanapp_id': 194,
   'result': True,
   'error_reason': 'no reasons',
   'detail_results': None,
   'fico_v2': '695',
   'report_type': 'CreditProfile',
   'username': 'cho',
   'created_at': '2016-08-04T00:29:03.067Z',
   'xml_data': '<?xml version="1.0" standalone="no"?><NetConnectResponse xmlns="http://www.experian.com/NetConnectResponse"><CompletionCode>0000</CompletionCode><ReferenceId>testing 123</ReferenceId><TransactionId>41588980</TransactionId><Products xmlns="http://www.experian.com/ARFResponse"><CreditProfile><Header><ReportDate>10032017</ReportDate><ReportTime>140607</ReportTime><Preamble>TWA1</Preamble><ARFVersion>07</ARFVersion></Header><RiskModel><ModelIndicator code="F "/><Score>0808</Score><ScoreFactorCodeOne>10</ScoreFactorCodeOne><ScoreFactorCodeTwo>09</ScoreFactorCodeTwo><ScoreFactorCodeThree>05</ScoreFactorCodeThree><ScoreFactorCodeFour>  </ScoreFactorCodeFour><Evaluation code="P"/></RiskModel><RiskModel>

Parse XML data string and build the internal objects into the designed tables

In [162]:
# initialize result object lists
RESULT_OBJ_DICT = dict((key, []) for key in RESULT_TABLE_NAMES)
ATTR_CODE_DICT = {
    'risk_models': ['ModelIndicator', 'Evaluation'],
    'addr_infos': ['Origination', 'DwellingType', 'HomeOwnership'],
    'employ_infos': ['Origination'],
    'trade_lines': ['SpecialComment', 'Evaluation', 'AccountType', 'TermsDuration', 'Status', 'ECOA', 'OpenOrClosed', 'RevolvingOrInstallment', 'MonthlyPaymentType', 'KOB'],
    'tradeline_amounts': ['Qualifier'],
    'enhanced_pm_data': ['AccountCondition', 'PaymentStatus', 'AccountType', 'SpecialComment'],
    'inquiries': ['Type', 'Terms', 'KOB']
}

prequal_results = json_file
for prequal_result in prequal_results:
    prequal_result.update(prequal_result.pop('fields'))
    xml_detail_id = "xd_{}".format(uuid.uuid4())
    prequal_result['xml_detail_id'] = xml_detail_id
    RESULT_OBJ_DICT['prequal_results'].append(prequal_result)
    
    # initializing XML_Detail object
    xml_dict = xmltodict.parse(result['xml_data'], dict_constructor=dict)['NetConnectResponse']
    xml_dict['id'] = xml_detail_id
    RESULT_OBJ_DICT['xml_details'].append(xml_dict)
    product_id = "pd_{}".format(uuid.uuid4())
    xml_dict['product_id'] = product_id
    
    # initializing Product object
    product_obj = xml_dict.pop('Products')
    product_obj['id'] = product_id
    RESULT_OBJ_DICT['products'].append(product_obj)
    credit_profile_id = "cp_{}".format(uuid.uuid4())
    product_obj['credit_profile_id'] = credit_profile_id
    
    # initializing CreditProfile object
    credit_profile_obj = product_obj.pop('CreditProfile')
    credit_profile_obj['id'] = credit_profile_id
    RESULT_OBJ_DICT['credit_profiles'].append(credit_profile_obj)
    
    
    ## handling RiskModels in CreditProfile
    for risk_model in credit_profile_obj.pop('RiskModel'):
        risk_model['id'] = "rm_{}".format(uuid.uuid4())
        risk_model['CreditProfile_id'] = credit_profile_id
        RESULT_OBJ_DICT['risk_models'].append(risk_model)
    
    # initializing Header object in CreditProfile
    header_id = "hd_{}".format(uuid.uuid4())
    credit_profile_obj['header_id'] = header_id
    header_obj = credit_profile_obj.pop('Header')
    header_obj['id'] = header_id
    RESULT_OBJ_DICT['headers'].append(header_obj)
    
    ## handling ConsumerIdentities in CreditProfile
    for consumer_identity in credit_profile_obj.pop('ConsumerIdentity'):
        consumer_identity['id'] = "ci_{}".format(uuid.uuid4())
        consumer_identity['CreditProfile_id'] = credit_profile_id
        RESULT_OBJ_DICT['consumer_identities'].append(consumer_identity)
    
    ## handling AddressInformations in CreditProfile
    for addr_info in credit_profile_obj.pop('AddressInformation'):
        addr_info['id'] = "ai_{}".format(uuid.uuid4())
        addr_info['CreditProfile_id'] = credit_profile_id
        RESULT_OBJ_DICT['addr_infos'].append(addr_info)
       
    ## handling EmploymentInformations in CreditProfile
    for employ_info in credit_profile_obj.pop('EmploymentInformation'):
        employ_info['id'] = "ei_{}".format(uuid.uuid4())
        employ_info['CreditProfile_id'] = credit_profile_id
        RESULT_OBJ_DICT['employ_infos'].append(employ_info)
        
    ## handling TradeLines in CreditProfile
    code_attrs = ['SpecialComment', 'Evaluation', 'AccountType', 'TermsDuration', 'Status', 'OpenOrClosed', 'RevolvingOrInstallment', 'MonthlyPaymentType', 'KOB']
    for trade_line in credit_profile_obj.pop('TradeLine'):
        trade_line['id'] = "tl_{}".format(uuid.uuid4())
        trade_line['CreditProfile_id'] = credit_profile_id
        RESULT_OBJ_DICT['trade_lines'].append(trade_line)
        
        # initializing EnhancedPaymentData object in TradeLine
        enhanced_pm_data_id = "ep_{}".format(uuid.uuid4())
        trade_line['enhanced_pm_data_id'] = enhanced_pm_data_id
        enhanced_pm_data_obj = trade_line.pop('EnhancedPaymentData')
        enhanced_pm_data_obj['id'] = enhanced_pm_data_id
        RESULT_OBJ_DICT['enhanced_pm_data'].append(enhanced_pm_data_obj)
        
        ## handling Amount object in TradeLine
        for amt_obj in trade_line.pop('Amount'):
            amt_obj['id'] = "am_{}".format(uuid.uuid4())
            amt_obj['TradeLine_id'] = trade_line['id']
            RESULT_OBJ_DICT['tradeline_amounts'].append(amt_obj)
    
    # initializing Inquiry object in CreditProfile
    inquiry_id = "iq_{}".format(uuid.uuid4())
    credit_profile_obj['inquiry_id'] = inquiry_id
    inquiry_obj = credit_profile_obj.pop('Inquiry')
    inquiry_obj['id'] = inquiry_id
    RESULT_OBJ_DICT['inquiries'].append(inquiry_obj)
    
    ## handling InformationalMessages in CreditProfile
    for info_msg in credit_profile_obj.pop('InformationalMessage'):
        info_msg['id'] = "im_{}".format(uuid.uuid4())
        info_msg['CreditProfile_id'] = credit_profile_id
        RESULT_OBJ_DICT['info_msgs'].append(info_msg)
    
# converting sub-attrs into _code
for attr, code_attr_list in ATTR_CODE_DICT.items():
    attr_obj_list = RESULT_OBJ_DICT[attr]
    for attr_obj in attr_obj_list:
        for attr in code_attr_list:
            if attr in attr_obj:
                code = attr_obj[attr]['@code']
            else:
                code = ''
            attr_obj["{}_code".format(attr)] = code
            attr_obj.pop(attr, None)
    
RESULT_OBJ_DICT

{'prequal_results': [{'model': 'common.prequalresult',
   'pk': 1,
   'loanapp_id': 194,
   'result': True,
   'error_reason': 'no reasons',
   'detail_results': None,
   'fico_v2': '695',
   'report_type': 'CreditProfile',
   'username': 'cho',
   'created_at': '2016-08-04T00:29:03.067Z',
   'xml_data': '<?xml version="1.0" standalone="no"?><NetConnectResponse xmlns="http://www.experian.com/NetConnectResponse"><CompletionCode>0000</CompletionCode><ReferenceId>testing 123</ReferenceId><TransactionId>41588980</TransactionId><Products xmlns="http://www.experian.com/ARFResponse"><CreditProfile><Header><ReportDate>10032017</ReportDate><ReportTime>140607</ReportTime><Preamble>TWA1</Preamble><ARFVersion>07</ARFVersion></Header><RiskModel><ModelIndicator code="F "/><Score>0808</Score><ScoreFactorCodeOne>10</ScoreFactorCodeOne><ScoreFactorCodeTwo>09</ScoreFactorCodeTwo><ScoreFactorCodeThree>05</ScoreFactorCodeThree><ScoreFactorCodeFour>  </ScoreFactorCodeFour><Evaluation code="P"/></RiskModel>

Take a preview of table PrequalResult

In [151]:
pd.DataFrame(RESULT_OBJ_DICT['prequal_results'])

,model,pk,loanapp_id,result,error_reason,detail_results,fico_v2,report_type,username,created_at,...,business_credit_card_total_limits,business_credit_card_total_balances,business_total_mortgage_monthly_payments,business_total_monthly_debt_payments,bkdate,fico_v3,total_mortgage_monthly_payments,version,role,xml_detail_id
0,common.prequalresult,1,194,True,no reasons,None,695,CreditProfile,cho,2016-08-04T00:29:03.067Z,...,None,None,None,None,None,None,None,CF Microloan,applicant,xd_eb7c47ff-8048-441a-b153-522643657d03


Take a preview of table Product

In [153]:
pd.DataFrame(RESULT_OBJ_DICT['products'])

,@xmlns,id,credit_profile_id
0,http://www.experian.com/ARFResponse,pd_82762c24-7362-4c6d-a95c-01da3be29db0,cp_83188663-1c2f-41ae-8fed-2e46b31ecaea


Take a preview of table CreditProfile

In [154]:
pd.DataFrame(RESULT_OBJ_DICT['credit_profiles'])

,id,header_id,inquiry_id
0,cp_83188663-1c2f-41ae-8fed-2e46b31ecaea,hd_530c0a97-f26c-4d67-b153-3b11af4b1cea,iq_e15be34d-446d-4998-b652-b8e98fcedd46


Take a preview of table Header

In [155]:
pd.DataFrame(RESULT_OBJ_DICT['headers'])

,ReportDate,ReportTime,Preamble,ARFVersion,id
0,10032017,140607,TWA1,07,hd_530c0a97-f26c-4d67-b153-3b11af4b1cea


Take a preview of table RiskModel

In [156]:
pd.DataFrame(RESULT_OBJ_DICT['risk_models'])

,Score,ScoreFactorCodeOne,ScoreFactorCodeTwo,ScoreFactorCodeThree,ScoreFactorCodeFour,id,CreditProfile_id,ModelIndicator_code,Evaluation_code
0,0808,10,09,05,None,rm_3f81f6bf-6b10-47f4-b21c-06bd3242fec8,cp_83188663-1c2f-41ae-8fed-2e46b31ecaea,F,P
1,0807,30,05,11,None,rm_17da3e39-4320-4f1e-a642-7f3da2160e71,cp_83188663-1c2f-41ae-8fed-2e46b31ecaea,AA,P


Take a preview of table ConsumerIdentity

In [157]:
pd.DataFrame(RESULT_OBJ_DICT['consumer_identities'])

,Name,YOB,id,CreditProfile_id
0,"{'Surname': 'SCALICI', 'First': 'WILLIAM'}",None,ci_7a8676e3-94f3-49e5-9f67-a224ba45a787,cp_83188663-1c2f-41ae-8fed-2e46b31ecaea
1,"{'Type': {'@code': 'N'}, 'Surname': 'SCALICI',...",None,ci_0e5ca0cd-0121-438d-a996-59c277725087,cp_83188663-1c2f-41ae-8fed-2e46b31ecaea
2,"{'Surname': 'SCALICI', 'First': 'WILLIAM', 'Ge...",None,ci_260f4c36-42b7-4b0b-901c-1699835bc99c,cp_83188663-1c2f-41ae-8fed-2e46b31ecaea
3,"{'Type': {'@code': 'A'}, 'Surname': 'ANTHONY',...",None,ci_9f9a2e62-5a73-4726-b9d1-9a90181964ff,cp_83188663-1c2f-41ae-8fed-2e46b31ecaea


Take a preview of table AddressInfo

In [158]:
pd.DataFrame(RESULT_OBJ_DICT['addr_infos'])

,FirstReportedDate,LastUpdatedDate,TimesReported,LastReportingSubcode,StreetPrefix,StreetName,StreetSuffix,City,State,Zip,CensusGeoCode,id,CreditProfile_id,Origination_code,DwellingType_code,HomeOwnership_code
0,10272003,12202015,00,7370142,7324,84TH STREET,CT SW,TACOMA,WA,984986380,None,ai_8a249318-d504-4814-b0d2-28e656b033e2,cp_83188663-1c2f-41ae-8fed-2e46b31ecaea,2,S,
1,01012011,01012011,00,None,3260 S,TACOMA,WAY,TACOMA,WA,984094725,None,ai_180a3b79-fc80-4969-91ea-4568b16cd4b5,cp_83188663-1c2f-41ae-8fed-2e46b31ecaea,1,S,
2,05282006,11022010,12,None,3216 S,TACOMA,WAY,TACOMA,WA,984094725,None,ai_794d9a61-3eae-405d-b469-1dc26e2a8954,cp_83188663-1c2f-41ae-8fed-2e46b31ecaea,1,S,


Take a preview of table EmploymentInfo

In [159]:
pd.DataFrame(RESULT_OBJ_DICT['employ_infos'])

,FirstReportedDate,LastUpdatedDate,Name,AddressFirstLine,AddressSecondLine,AddressExtraLine,Zip,id,CreditProfile_id,Origination_code
0,04082011,04082011,GM RETIRED,None,None,None,None,ei_da29547c-22e3-455b-a51f-79af941f6917,cp_83188663-1c2f-41ae-8fed-2e46b31ecaea,2
1,12282009,06112010,RETIRED,None,None,None,None,ei_7fe91e47-c26a-4527-96e1-90ec73229d55,cp_83188663-1c2f-41ae-8fed-2e46b31ecaea,2


Take a preview of table TradeLine

In [165]:
pd.DataFrame(RESULT_OBJ_DICT['trade_lines'])

,OpenDate,StatusDate,MaxDelinquencyDate,BalanceDate,BalanceAmount,AmountPastDue,ConsumerComment,MonthsHistory,DelinquenciesOver30Days,DelinquenciesOver60Days,...,Evaluation_code,AccountType_code,TermsDuration_code,Status_code,ECOA_code,OpenOrClosed_code,RevolvingOrInstallment_code,MonthlyPaymentType_code,KOB_code,AccountNumber
0,06011981,05012017,None,05192017,None,None,None,15,00,00,...,N,18,REV,05,1,C,R,,OC,NaN
1,08282007,03012012,None,03182012,None,None,None,56,00,00,...,N,18,REV,05,1,C,R,,BC,NaN
2,06102006,07012017,None,07232017,00000000,None,None,96,00,00,...,N,18,REV,11,2,C,R,,BC,NaN
3,08282007,03012012,None,04282012,00000000,None,None,03,00,00,...,N,18,REV,11,1,C,R,,BC,NaN
4,03012002,04012009,None,04012009,00000000,None,None,86,00,00,...,N,18,REV,11,0,C,R,,NZ,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,06011983,08012017,None,08072017,00000052,None,None,01,00,00,...,P,18,REV,11,1,O,R,,BC,NaN
32,05011984,08012017,None,08072017,00000599,None,None,01,00,00,...,P,18,001,11,1,O,R,,BC,NaN
33,05011984,08012017,None,08072017,00001960,None,None,01,00,00,...,P,18,REV,11,1,O,R,,BC,NaN
34,08182016,08012017,None,08072017,00000000,None,None,12,00,00,...,P,07,REV,11,1,O,R,,CG,NaN


Take a preview of table Inquiry

In [166]:
pd.DataFrame(RESULT_OBJ_DICT['inquiries'])

,Date,Amount,Subcode,SubscriberDisplayName,id,Type_code,Terms_code,KOB_code
0,08182016,UNKNOWN,1640007,SYNCB/BROOKS BROS,iq_3fc54910-a551-4314-85d9-51c8d0d5cfec,31,UNK,FF


Take a preview of table InformationMessage

In [167]:
pd.DataFrame(RESULT_OBJ_DICT['info_msgs'])

,MessageNumber,MessageText,id,CreditProfile_id
0,92,0092 REQUESTED PRODUCT OPTION NOT ALLOWED,im_28268464-7836-441b-a0d0-18f3205ef32c,cp_0160ed5d-ab9c-42b1-8f2a-7b438e70b0a6
1,84,0084 SSN MATCHES,im_ebdda4fd-48a0-41fc-93db-5e2749d0ba4f,cp_0160ed5d-ab9c-42b1-8f2a-7b438e70b0a6


In [ ]:
Take a preview of table EnhancedPaymentData

In [168]:
pd.DataFrame(RESULT_OBJ_DICT['enhanced_pm_data'])

,InitialPaymentLevelDate,id,AccountCondition_code,PaymentStatus_code,AccountType_code,SpecialComment_code
0,05012017,ep_3c421ba6-2346-428c-8d27-68fe80c0d68c,05,11,18,51
1,03012012,ep_568ec0f9-f267-4dc6-9a1a-a900a502f03b,05,11,18,40
2,07012017,ep_047b1fda-32da-4a2f-9132-2b319fafbd66,A3,11,18,19
3,03012012,ep_1d2705f9-1a92-478d-b1d2-6a116354c0c6,A3,11,18,18
4,04012009,ep_67ca52d4-0e3f-4043-acb4-4e35c00b3135,A3,11,18,19
...,...,...,...,...,...,...
31,08012017,ep_0cd98c68-209d-46ed-813c-b46e6b79dc19,A1,11,18,
32,08012017,ep_5f1be051-aa86-49ed-ad57-a2fad69d656b,A1,11,18,
33,08012017,ep_65e68832-a111-4c24-9f1b-3f1cb61cbcbd,A1,11,18,
34,08012017,ep_6dfac92c-1023-4fc9-9852-105664319d7e,A1,11,07,


In [ ]:
Take a preview of table EnhancedPaymentData

In [ ]:
pd.DataFrame(RESULT_OBJ_DICT['tradeline_amounts'])